<a href="https://colab.research.google.com/github/schmellerl/gradient_flows_order_parameters_mechanics/blob/main/colab/Example3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# put in a separate file
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/ColabNumerics/Example3/'

<h1>Gradient flows for coupling order parameters and mechanics</h1>

FENICS implementation of the examples from

Schmeller, L. & Peschka, D. (2021). arXiv:XXXX.YYYY


DOI:

<h3>General coupled gradient flow evolution</h3>

For a state variable $q=(u,\psi)$ with displacement $u:\Omega\to\mathbb{R}^d$ and phase field $\psi:\Omega\to\mathbb{R}^N$ we consider the following free energy 
\begin{align}
    \mathscr{F}(q) 
    = & 
    \int_{{\Omega}}W_{\rm elast}(F_e,\psi){\rm d}x
    + \int_{{\Omega}}W_{\rm phase}(\psi,\nabla\psi,F)
    {\rm d}x
\end{align}

for deformation gradient $F=\mathbb{I}_d+\nabla u=F_e F_p$ and with given plastic strain $F_p=F_p(\psi)$ and $F_e=FF_p^{-1}$.

We consider a Neo-Hookean elastic energy density 
\begin{align}
   W_{\rm elast}(F_e,\psi) = \frac{G}{2}\left(\text{tr}(F_e^\top F_e - \mathbb{I}_d) -2\log(\det(F_e))\right)+
 \frac{K}{2}\big(\det(F_e)-H\big)^2  
\end{align}
with bulk modulus $G=G(\psi)$ and phase volume $H=H(\psi)$ and inverse compressibility $K\in\mathbb{R}$. The remaining part of the free energy is 
\begin{align}
    W_\text{phase}(\psi,\nabla\psi,F) &= \left[\frac{1}{2}\nabla\psi\cdot \sigma\nabla\psi + W_\text{entropy}(\psi,F)\right]\det(F)
\end{align}
with second-order tensor $\sigma=\sigma(\psi,F)\in\mathbb{R}^{d\times d}$. 
Together with a given dissipation potential $R(q,\dot{q})$, different parameters, double-well or Flory-Huggins-type entropy $W_\text{entropy}$ and possible constraints $C(q,\nabla q)=0\in\mathbb{R}^M$ added through the Lagrangian in terms of $q_\lambda=(q,\lambda)$ with the optional Lagrange multiplier $\lambda:\Omega\to\mathbb{R}^M$
\begin{align}
\mathscr{L}(q_\lambda)=\mathscr{F}(q)+\int_\Omega C(q,\nabla q)\cdot\lambda\,{\rm d}x
\end{align}
we consider the formal gradient flow evolution $\partial_t q =-\nabla_R \mathscr{F}(q)$, which we solve by a saddle-point problem generated by the minimization problem
\begin{align}
%\partial_t q =-\nabla_R \mathscr{L}(q)
%\quad\Leftrightarrow\quad \left(
  \min_{v_\lambda=(v,\hat{\lambda})} \Big[R(q,v)+\langle \mathrm{D}\mathscr{L}(q_\lambda),v_\lambda\rangle\Big]%\right)
\end{align}
solved by incremental minimization as described in more detail in the manuscript.

In [ ]:
# put in a separate file
from IPython.display import HTML, display
def progress(value, max=100):
    return HTML("""
        Iterations: {value} of {max}
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

from fenics import *
from matplotlib import pyplot as plt
import numpy as np
import random
from mshr import *

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams.update({'font.size': 14})

# should not appear in public version
def output_mesh(mesh):
    File(filepath + "mesh.xml") << mesh

def output_solution(q,n,t=0):
    u, psi1, psi2, *_ = q.split()

    File(filepath + "u"+str(n)+".xml") << project(q.sub(0),FunctionSpace(mesh,Vu)) 
    File(filepath + "psi1"+str(n)+".xml") << project(q.sub(1),FunctionSpace(mesh,Vpsi))
    File(filepath + "psi2"+str(n)+".xml") << project(q.sub(2),FunctionSpace(mesh,Vpsi))
    #File(filepath + "eta_psi"+str(n)+".xml") << project(q.sub(2),FunctionSpace(mesh,U))


# Example 1:

For the simplest example we consider a fixed two-dimensional domain $\Omega=(0,1)^2$ with homogeneous Dirichlet boundary conditions $u=0$ on $\partial\Omega$. We use one species $N=1$ and other constitutive relations / paramters

\begin{align}
G&=10\\
K&=10\\
H&=1\\
\sigma&=\varepsilon F^{-1}F^{-\top} \qquad \varepsilon=0.04\\
F_p&=(1+\alpha\psi)\mathbb{I}_2 \qquad \alpha=0.5 \\
W_\text{entropy}&=\frac{1}{2\varepsilon}(1-\psi^2)^2\\
R(q,\nabla q)&=\text{Cahn-Hilliard-type }R_\psi\text{ with }\mu=1\\
C(q,\nabla q)&=\text{no constraints in this example}
\end{align}

In [ ]:
# model parameters
N_T   = 1000            # time steps
G1    = Constant(1000)  # elastic modulus phase 1
G2    = 0.3*G1          # elastic modulus phase 2
G3    = 0.3*G1          # elastic modulus phase 3

def GshearF(psi1,psi2):
    psi3 = -1-psi1-psi2
    xi1 = (1+psi1)/2
    xi2 = (1+psi2)/2
    xi3 = (1+psi3)/2
    return  (xi1*G1 + xi2*G2 + xi3*G3)

# surface tension
gamma1  = 1e-3    # solig
gamma2  = 6e-3    # liquid
gamma3  = 24e-3   # air 

mu    = Constant(0.001)    # Cahn-Hilliard mobility
nu    = Constant(0.01)     # Solid mobility  

h0    = 4.0e-6 # substrate height 
h1    = 2.0e-6 # fluid height

#eps   = 6e-9
interface_factor = Constant(1.0/sqrt(2))


# mesh parameters of tensorial Nx x Nx x 2 triangular elements mesh
H0    = 0
#H1    = 3e-6    # width of domain
#H2    = 7e-6    # height of domain

# nondimensional parameters
h0T   = h0/h0 
h1T   = h1/h0
eps   = 6e-9/h0
H1    = 4e-6/h0
H2    = 8e-6/h0

LL    = 0 # 0 or H1 (rotational symmetry or 2d)

# mesh
Nx    = 2
Ny    = 4
mesh  = RectangleMesh(Point(H0,H0),Point(H1,H2),Nx,Ny)

# define the function spaces
Vu    = VectorElement("P", mesh.ufl_cell(), 1) # V_u   = displacements
Vpsi  = FiniteElement("P", mesh.ufl_cell(), 1) # V_psi = order-parameter(s)
Uu    = VectorElement("P", mesh.ufl_cell(), 1)
Upsi  = FiniteElement("P", mesh.ufl_cell(), 1) # U     = forces
#VxU   = FunctionSpace(mesh, MixedElement([Vu,Vpsi,U])) # tensor space of V x U
R     = FiniteElement("P", mesh.ufl_cell(), 1) 

VxUxR = FunctionSpace(mesh, MixedElement([Vu,Vpsi,Vpsi,Uu,Upsi,Upsi,R])) 

# homogeneous Dirichlet boundary conditions 
noslip    = Constant((0, 0)) 
noslip1d  = Constant(0) 

def boundary_bot1(x, on_boundary1):
    tol = 1E-14
    return on_boundary1 and near(x[0], H0, tol)

def boundary_bot2(x, on_boundary2):
    tol = 1E-14
    return on_boundary2 and near(x[1], H0, tol)

def boundary_bot3(x, on_boundary3):
    tol = 1E-14
    return on_boundary3 and near(x[0], H1, tol)

def boundary_bot4(x, on_boundary4):
    tol = 1E-14
    return on_boundary4 and near(x[1], H2, tol)

 
bc1   = DirichletBC(VxUxR.sub(0).sub(0), noslip1d, boundary_bot1)
bc2   = DirichletBC(VxUxR.sub(0), noslip, boundary_bot2)
bc3   = DirichletBC(VxUxR.sub(0).sub(0), noslip1d, boundary_bot3)
bc4   = DirichletBC(VxUxR.sub(0), noslip, boundary_bot4)

In [ ]:
# incremental minimization for coupled finite strain elasticity + phase field
def incremental_minimization(old_q, tau):
    
    q  = Function(VxUxR)   
    dq = TestFunction(VxUxR)

    # current solution
    u, psi1, psi2, eta_u, eta_psi1, eta_psi2, lambda1 = split(q)
    # old solution
    old_u,old_psi1,old_psi2,*_  = split(old_q)
    # test functions
    du,dpsi1,dpsi2,deta_u, deta_psi1, deta_psi2,_  = TestFunctions(VxUxR)

    # define continuum mechanics / plasticity variables
    d       = psi1.geometric_dimension()
    I       = Identity(d)
    F       = I + grad(u)       # deformation gradient
    J       = det(F)            # Jacobian strain
    C       = F.T*F             # stress tensor

    psi3 = -1-psi1-psi2
    
    gradpsi1 = inv(F).T*grad(psi1) 
    gradpsi2 = inv(F).T*grad(psi2)  
    gradpsi3 = inv(F).T*grad(psi3)  
    
    # define free energy F_free 
    W_elastic   = (GshearF(psi1,psi2)/2)/G1*(tr(C - I) )
    W_phase     = gamma1/G1*1/h0*( 1/(4*eps)*(1-(psi1)**2)**2 + (eps/2)*inner(gradpsi1,gradpsi1) )*J
    W_phase    += gamma2/G1*1/h0*( 1/(4*eps)*(1-(psi2)**2)**2 + (eps/2)*inner(gradpsi2,gradpsi2) )*J
    W_phase    += gamma3/G1*1/h0*( 1/(4*eps)*(1-(psi3)**2)**2 + (eps/2)*inner(gradpsi3,gradpsi3) )*J
    
    c1          = (J-1)

    r   =  Expression('x[0]', degree = 2) # Constant(1.0)

    F_free    = (W_elastic + W_phase)*r*dx

    L         = F_free + c1*lambda1*r*dx

    # backward Euler time derivative
    dot_u    = (u-old_u)/tau
    dot_psi1 = (psi1-old_psi1)/tau
    dot_psi2 = (psi2-old_psi2)/tau

    # add energy and M_psi operator
    Res  = derivative(L, q, dq) - inner(eta_u,du)*r*dx - inner(eta_psi1,dpsi1)*r*dx - inner(eta_psi2,dpsi2)*r*dx
        
    # add bilinear form a_psi and M_psi*
    Res += inner(dot_u   , deta_u   )*r*dx   + inner(nu*eta_u      , deta_u          )*r*dx 
    Res += inner(dot_psi1, deta_psi1)*r*dx   + inner(mu*inv(F).T*grad(eta_psi1), inv(F).T*grad(deta_psi1) )*r*dx 
    Res += inner(dot_psi2, deta_psi2)*r*dx   + inner(mu*inv(F).T*grad(eta_psi2), inv(F).T*grad(deta_psi2) )*r*dx
     
    
    # solve nonlinear problem using old solution as initial guess
    q.assign(old_q)    
    parameters["form_compiler"]["cpp_optimize"] = True
    parameters["form_compiler"]["quadrature_degree"] = 5

    bc1   = DirichletBC(VxUxR.sub(0).sub(0), noslip1d, boundary_bot1)
    bc2   = DirichletBC(VxUxR.sub(0), noslip, boundary_bot2)
    bc3   = DirichletBC(VxUxR.sub(0).sub(0), noslip1d, boundary_bot3)
    bc4   = DirichletBC(VxUxR.sub(0), noslip, boundary_bot4)

    bc = [bc1,bc2,bc3,bc4]

    solve(Res == 0, q, bc)


    E_free    = assemble(F_free)

    return q,E_free         

In [ ]:
def mesh_space(mesh,VxUxR,Upsi,q):
    
    ep = 0.13*6
  
    u, psi1, psi2, eta_u, eta_psi1, eta_psi2, lambda1 = split(q)
    
    #tmp = Function(FunctionSpace(mesh, Upsi))
    tmp = project(psi1,FunctionSpace(mesh, Upsi)) 

    #tmp2 = Function(FunctionSpace(mesh, Upsi))
    tmp2 = project(psi2,FunctionSpace(mesh, Upsi))
    
    cell_markers = MeshFunction("bool", mesh,2)
    cell_markers.set_all(False)

    qTmp = tmp.vector()
    qTmp2 = tmp2.vector()
    
    qMin = Function(FunctionSpace(mesh, Upsi))
    qMax = Function(FunctionSpace(mesh, Upsi))
    
    qMin2 = Function(FunctionSpace(mesh, Upsi))
    qMax2 = Function(FunctionSpace(mesh, Upsi))
    
    
    qTMin = qMin.vector()
    qTMax = qMax.vector()
    
    qTMin2 = qMin.vector()
    qTMax2 = qMax.vector()

        
    dm = FunctionSpace(mesh, Upsi).dofmap()
    
     
    for cell in cells(mesh):
         
        cell_index = cell.index()

        
        cell_dofs = dm.cell_dofs(cell_index)
                           
                                  
        a = qTmp[cell_dofs].min()
        b = qTmp[cell_dofs].max()
    
        a2 = qTmp2[cell_dofs].min()
        b2 = qTmp2[cell_dofs].max()

        
        if (a-ep<0) and (b+ep>0):
            cell_markers[cell] = True
            
        if (a2-ep<0) and (b2+ep>0):
            cell_markers[cell] = True
                            
                       
    mesh = refine(mesh, cell_markers)
    
    Vu    = VectorElement("P", mesh.ufl_cell(), 1) # V_u   = displacements
    Vpsi  = FiniteElement("P", mesh.ufl_cell(), 1) # V_psi = order-parameter(s)
    Uu    = VectorElement("P", mesh.ufl_cell(), 1)
    Upsi  = FiniteElement("P", mesh.ufl_cell(), 1) # U     = forces
    #VxU   = FunctionSpace(mesh, MixedElement([Vu,Vpsi,U])) # tensor space of V x U
    R     = FiniteElement("P", mesh.ufl_cell(), 1) 
    
    VxUxR = FunctionSpace(mesh, MixedElement([Vu,Vpsi,Vpsi,Uu,Upsi,Upsi,R])) 
    
    # Boundary

    bc1   = DirichletBC(VxUxR.sub(0).sub(0), noslip1d, boundary_bot1)
    bc2   = DirichletBC(VxUxR.sub(0), noslip, boundary_bot2)
    bc3   = DirichletBC(VxUxR.sub(0).sub(0), noslip1d, boundary_bot3)
    bc4   = DirichletBC(VxUxR.sub(0), noslip, boundary_bot4)

    bc = [bc1,bc2,bc3,bc4]
    
    #bc=[DirichletBC(W.sub(0), noslip, boundary_bot),DirichletBC(W.sub(3), noslip, boundary_bot)]
    
    return bc, VxUxR, Vpsi, mesh


In [ ]:
initial = Expression(("0",
                      "0",
                      "tanh(ff*(h0-x[1])/eps)",
                      "-1 + 2*(1+tanh(ff*(x[1]/h0-h0)/eps))*(1+tanh(ff/eps*(h1-pow(pow((x[0])/h0-LL/2,2.0)+pow(x[1]/h0-h0,2.0),0.5))))/4",
                      "0","0","0","0","0"), eps=eps,h0=h0T,h1=h1T,degree=2,ff=interface_factor,LL=LL)



mesh  = RectangleMesh(Point(H0,H0),Point(H1,H2),Nx,Ny)

###########
## unrefined mesh with function spaces 
Vu    = VectorElement("P", mesh.ufl_cell(), 1)
Vpsi  = FiniteElement("P", mesh.ufl_cell(), 1) 
Uu    = VectorElement("P", mesh.ufl_cell(), 1)
Upsi  = FiniteElement("P", mesh.ufl_cell(), 1)   
R     = FiniteElement("P", mesh.ufl_cell(), 1) 
###########

VxUxR = FunctionSpace(mesh, MixedElement([Vu,Vpsi,Vpsi,Uu,Upsi,Upsi,R])) 


old_q = interpolate(initial, VxUxR)

out   = display(progress(0, N_T), display_id=True)

#tau             = 1e-6

bc, VxUxR, Vpsi, mesh  = mesh_space(mesh,VxUxR,Vpsi,old_q)
old_q           = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, mesh  = mesh_space(mesh,VxUxR,Vpsi,old_q)
old_q           = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, mesh  = mesh_space(mesh,VxUxR,Vpsi,old_q)
old_q           = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, mesh  = mesh_space(mesh,VxUxR,Vpsi,old_q)
old_q           = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, mesh  = mesh_space(mesh,VxUxR,Vpsi,old_q)
old_q           = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, mesh  = mesh_space(mesh,VxUxR,Vpsi,old_q)
old_q           = interpolate(initial, VxUxR)
bc, VxUxR, Vpsi, mesh  = mesh_space(mesh,VxUxR,Vpsi,old_q)
old_q           = interpolate(initial, VxUxR)

#q,E_free = incremental_minimization(old_q, tau) 

energies     = [[0,0.0,E_free]]
#old_q.assign(q)

t   = 0

output_mesh(mesh)
output_solution(q,0,t)

for n in range(N_T) :
  
    if n<10:
      tau = 0.001 / N_T * (1/10)
    else:
      tau = 0.001 / N_T

    t += tau
    q,E_free = incremental_minimization(old_q, tau) 
    energies.append([n,t,E_free])
    output_solution(q,n+1,t)
    out.update(progress( n+1 , N_T )) 
    old_q.assign(q)

E = list(zip(*energies))
np.save(filepath + "energies",E)

In [ ]:
# put in a separate file
u, psi1, psi2, *_ = split(q)

psi3 = project(-1-psi1-psi2,FunctionSpace(mesh, Vpsi))
psi4 = project( (1+psi1)/2 + 2*(1+psi2)/2 + 3*(1+psi3)/2 ,FunctionSpace(mesh, Vpsi))

E = list(zip(*energies))
np.save(filepath + "energies",E)

plt.figure(figsize=(2*20,20))
plt.subplot(1,3,1)
A = plot(psi4)
plt.colorbar(A)
#plot(u)
#plt.title("$u$ and $\psi$")
plt.title("$\psi$")

ALE.move(mesh,q.sub(0))
plt.subplot(1,3,2)
plot(psi4)
plot(mesh, linewidth=0.5)
ALE.move(mesh,project(-q,VxUxR).sub(0))

plt.title("$\psi$ and mesh")
plt.subplot(1,3,3)
plt.plot(E[1],E[2])
plt.grid(True)
plt.xlabel("time $t$")
plt.ylabel("$\mathscr{F}$")
plt.title("free energy vs time");
plt.savefig("test.jpeg")
